In [1]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import faiss
import os, io, pickle
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    SpacyTextSplitter
)
from langchain.vectorstores import FAISS
import numpy as np
import pandas as pd
import tiktoken
from sentence_transformers import SentenceTransformer
import shutil
import statistics

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def authenticate_google():
    creds = None
    if os.path.exists("token"):
        with open("token", "rb") as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials3.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token", "wb") as token:
            pickle.dump(creds, token)
    
    return build("drive", "v3", credentials=creds)

def download_pdf(file_id, output_path):
    service = authenticate_google()
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(output_path, "wb")
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f"Téléchargement : {int(status.progress() * 100)}%")
    print(f"Fichier téléchargé : {output_path}")


file_id = "1AHE1lXi_kyrtRw31qEGJ7OYE9EO8kfGe"
download_pdf(file_id, "Histoire_CM1.pdf")

Téléchargement : 100%
Fichier téléchargé : Histoire_CM1.pdf


In [3]:
# Fonction pour estimer le nombre de tokens d’un texte
def count_tokens(text, model="gpt-3.5-turbo"):
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text))

In [ ]:
# Charger ton PDF
loader = PyPDFLoader("Histoire_CM1.pdf",
                     mode = "page", # Extract the PDF by page. Each page is extracted as a langchain Document object
                     # mode = "single" # PyPDFLoader will split the PDF as a single text flow
                     )
docs = loader.load()

In [ ]:
full_text = "\n".join([page.page_content for page in docs])
full_text

'LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  \n \n \n1)  Qu’est-ce que l’Histoire ? \n \n*L’Histoire est l’étude de notre passé  pour mieux \ncomprendre notre vie aujourd’hui. \n \n *Pour découvrir notre passé, les historiens font des fouilles \narchéologiques, étudient des objets, des documents, des récits…  \n*Ils représentent le temps par une ligne graduée  : c’est la frise \nchronologique.  \n \n*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient \nl’Histoire.  \n*L’Histoire de France est divisée en 5 périodes : \nl’Antiquité – le Moyen Âge – les Temps Modernes – le XIX ème \nsiècle – le XXème siècle. \n \n \n \n \n2) Des traces du passé : les grottes ornées \n \n*En 1940, 4 enfants découvrent une grotte recouverte \nde peintures  : des taureaux, des cerfs, des chevaux…  : \nla grotte de Lascaux.  En datant les objets trouvés dans la grotte on \nsait qu’elle a été peinte il y a environ 17000 ans.  \n \n*En 1994, Jean -Marie Chauvet découvre une autre grotte pei

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_text(full_text)
token_counts = [count_tokens(chunk) for chunk in chunks]

print(f"Nombre de chunks : {len(chunks)}")
print(f"Tokens/chunk (moy) : {round(statistics.mean(token_counts))}")
print(f"Tokens max : {max(token_counts)}")
print(f"Aperçu du 1er chunk :\n{chunks[0]}")


Nombre de chunks : 27
Tokens/chunk (moy) : 144
Tokens max : 161
Aperçu du 1er chunk :
LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  
 
 
1)  Qu’est-ce que l’Histoire ? 
 
*L’Histoire est l’étude de notre passé  pour mieux 
comprendre notre vie aujourd’hui. 
 
 *Pour découvrir notre passé, les historiens font des fouilles 
archéologiques, étudient des objets, des documents, des récits…  
*Ils représentent le temps par une ligne graduée  : c’est la frise 
chronologique.  
 
*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient 
l’Histoire.


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)

print("Taille des embeddings : ", embeddings.shape)
for i, (chunk, emb) in enumerate(zip(chunks, embeddings)):
    print("="*60)
    print(f"Chunk {i+1}:")
    print(chunk)
    print(f"\n Embedding (taille {len(emb)}):")
    print(np.round(emb[:10], 3))  # Affiche les 10 premières valeurs, arrondies pour lisibilité
    print("="*60)

/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Taille des embeddings :  (27, 384)
Chunk 1:
LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  
 
 
1)  Qu’est-ce que l’Histoire ? 
 
*L’Histoire est l’étude de notre passé  pour mieux 
comprendre notre vie aujourd’hui. 
 
 *Pour découvrir notre passé, les historiens font des fouilles 
archéologiques, étudient des objets, des documents, des récits…  
*Ils représentent le temps par une ligne graduée  : c’est la frise 
chronologique.  
 
*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient 
l’Histoire.

 Embedding (taille 384):
[-0.024  0.15   0.059 -0.052 -0.017  0.151 -0.052  0.037 -0.035  0.003]
Chunk 2:
l’Histoire.  
*L’Histoire de France est divisée en 5 périodes : 
l’Antiquité – le Moyen Âge – les Temps Modernes – le XIX ème 
siècle – le XXème siècle. 
 
 
 
 
2) Des traces du passé : les grottes ornées 
 
*En 1940, 4 enfants découvrent une grotte recouverte 
de peintures  : des taureaux, des cerfs, des chevaux…  : 
la grotte de Lascaux.  En datant les objets trouvés d

In [ ]:
chroma_db_path = "chroma_db"
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
documents = [Document(page_content=chunk, metadata={"chunk_id": i}) for i, chunk in enumerate(chunks)]


chroma_db = Chroma.from_documents(
    documents, 
    embedding_model, 
    persist_directory = chroma_db_path)

chroma_db

/tmp/ipykernel_1840/448283172.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
# Test de recherche
query = "Qui est Napoléon ?"

# Récupère les chunks les plus pertinents associés à la requête
# Exemple avec une recherche basée sur un score de similarité
retriever = chroma_db.as_retriever(
    search_type = "similarity_score_threshold",
    search_kwargs = {"k": 3, "score_threshold": 0.2},
)

relevant_docs = retriever.invoke(query)
relevant_docs

/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[Document(id='c4fe1c91-5913-45da-8bb2-bbf3cb99d44b', metadata={'chunk_id': 23}, page_content='autre gouvernement est mis en place.  \n \n*Il sera renversé par Napoléon le 18 brumaire.   \n \n  *Constitution : texte qui précise comment un pays est dirigé.\n21) Le Consulat et l’Empire (1799-1815) \n \n*Le général Bonaparte remporte de nombreuses victoires \nmilitaires, comme Austerlitz.  \n \n*Il  adopte une nouvelle constitution qui lui donne tous les \npouvoirs.  \n \n*Il crée les départements dirigés par un préfet, la banque de France, \nles lycées…'),
 Document(id='523f3c46-204b-476d-9389-0258639d328a', metadata={'chunk_id': 23}, page_content='autre gouvernement est mis en place.  \n \n*Il sera renversé par Napoléon le 18 brumaire.   \n \n  *Constitution : texte qui précise comment un pays est dirigé.\n21) Le Consulat et l’Empire (1799-1815) \n \n*Le général Bonaparte remporte de nombreuses victoires \nmilitaires, comme Austerlitz.  \n \n*Il  adopte une nouvelle constitution qui lui 

In [ ]:
print("Nombre de chunks pertinents :", len(relevant_docs))
    
# Affiche les résultats pertinents avec les métadonnées associées
print("\n--- Documents les plus pertinents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"chunk_id : {doc.metadata.get('chunk_id')}\n")

Nombre de chunks pertinents : 3

--- Documents les plus pertinents ---
Document 1:
autre gouvernement est mis en place.  
 
*Il sera renversé par Napoléon le 18 brumaire.   
 
  *Constitution : texte qui précise comment un pays est dirigé.
21) Le Consulat et l’Empire (1799-1815) 
 
*Le général Bonaparte remporte de nombreuses victoires 
militaires, comme Austerlitz.  
 
*Il  adopte une nouvelle constitution qui lui donne tous les 
pouvoirs.  
 
*Il crée les départements dirigés par un préfet, la banque de France, 
les lycées…

chunk_id : 23

Document 2:
autre gouvernement est mis en place.  
 
*Il sera renversé par Napoléon le 18 brumaire.   
 
  *Constitution : texte qui précise comment un pays est dirigé.
21) Le Consulat et l’Empire (1799-1815) 
 
*Le général Bonaparte remporte de nombreuses victoires 
militaires, comme Austerlitz.  
 
*Il  adopte une nouvelle constitution qui lui donne tous les 
pouvoirs.  
 
*Il crée les départements dirigés par un préfet, la banque de France, 
les

# MON TEST

In [ ]:
context = chroma_db.get()
context

{'ids': ['019e4b05-7b38-47d8-8a67-896dac27b341',
  '6b1bc997-35d3-45e3-8554-21db3648153a',
  '8b143b7b-de44-472c-b3c8-fe0be0587c29',
  'dd56e79f-ffa9-4bd5-93cc-fdb1a4531f3b',
  'a04b7d64-d73b-4165-8bee-a64dd519ae5e',
  'd501bb50-8632-403e-8ca8-f063903cb0ae',
  'a4a2911a-9b88-400b-a9ba-87d3015f4c05',
  '89ea9b56-d4f8-482a-8f7b-6dfae78a7f65',
  'c0084a95-3052-41fc-8937-cd3508bd969d',
  '4ded77f0-d9c4-44c1-953e-1eb5ce29247c',
  '7acf8b9f-8577-4486-9c16-d2e9524c5931',
  'e2e05886-9674-461c-9d8d-c6889c7bb604',
  '6adadd40-f482-423f-8680-858721a170d6',
  '06c5336b-1dc3-4e39-a207-370b6c0279df',
  'a00ea27f-fb83-4cee-95b4-694b885f6150',
  '5b901de3-a9f8-4fa8-abc3-5d9434f7c4f3',
  '361d3dc8-e669-440e-a9c1-f8ca126c524d',
  'a60c59b4-d7ad-4ab6-bd74-ed377ced74bd',
  'a32c18f8-ead6-4b8d-92dc-59f8ca4a91c2',
  '8c22869e-adb0-43ff-9855-28002ca7dcf1',
  '0b14285b-37ac-4fb8-984d-e727723b4bd7',
  '5c57ce89-5059-47c2-a045-4de353776583',
  'e2084635-20b4-4710-ae96-c3eef48d2c6a',
  '0ad561de-e278-4f51-9e9a-

In [ ]:
prompt = f"""
Tu es un expert en pédagogie et tu dois créer une question de quiz à choix multiples (QCM) à partir du texte ci-dessous.

**Important** : tu ne dois utiliser que les informations contenues dans le texte ci-dessous.  
**Tu ne dois en aucun cas utiliser des connaissances extérieures.**

Contrainte :
- La question doit porter sur une information claire et importante du texte.
- Il doit y avoir exactement 4 choix.
- Une seule bonne réponse.
- Il faut que QCM porte uniquement
- Retourne ta réponse **en JSON valide** dans ce format :

{{
  "question": "...",
  "choices": ["...", "...", "...", "..."],
  "correct_answer": "..."
}}

Texte :
\"\"\"
{context["documents"]}
\"\"\"
"""

prompt

'\nTu es un expert en pédagogie et tu dois créer une question de quiz à choix multiples (QCM) à partir du texte ci-dessous.\n\n**Important** : tu ne dois utiliser que les informations contenues dans le texte ci-dessous.  \n**Tu ne dois en aucun cas utiliser des connaissances extérieures.**\n\nContrainte :\n- La question doit porter sur une information claire et importante du texte.\n- Il doit y avoir exactement 4 choix.\n- Une seule bonne réponse.\n- Il faut que QCM porte uniquement\n- Retourne ta réponse **en JSON valide** dans ce format :\n\n{\n  "question": "...",\n  "choices": ["...", "...", "...", "..."],\n  "correct_answer": "..."\n}\n\nTexte :\n"""\n[\'LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  \\n \\n \\n1)  Qu’est-ce que l’Histoire ? \\n \\n*L’Histoire est l’étude de notre passé  pour mieux \\ncomprendre notre vie aujourd’hui. \\n \\n *Pour découvrir notre passé, les historiens font des fouilles \\narchéologiques, étudient des objets, des documents, des récits…  \\n*Ils rep

In [ ]:
len(context['documents'])

351

In [ ]:
from langchain.llms import Ollama
llm = Ollama(model="mistral", base_url="http://host.docker.internal:11434")
response = llm.invoke("Génère une question de QCM sur les rois de France.")
print(response)

/tmp/ipykernel_1840/610573073.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral", base_url="http://host.docker.internal:11434")


 Quelle est la durée du règne du roi Philippe Auguste, le premier capétien à avoir réuni toutes les possessions de la couronne de France ?

A) 5 ans
B) 15 ans
C) 40 ans
D) 70 ans


In [ ]:
#from langchain.llms import Ollama
#llm = Ollama(model="mistral", base_url="http://host.docker.internal:11434")
#response2 = llm.invoke(prompt)
#print(response2)

In [ ]:
# NOTE
# Soit essayer une autre LLM que mistral via olamma
# Soit cleaner le code car le lancement serveur se fait dans le terminal ET le lancement de mistral ollama dans le terminal de la machine (et non dans le start)
# ollama serve -> terminal dev container
# ollama run mistral -> terminal machine

In [ ]:
from langchain.chat_models import ChatOpenAI
OPEN_ROUTER_KEY="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"

llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPEN_ROUTER_KEY,
    model="mistralai/mistral-7b-instruct"
)

response = llm.invoke(prompt)
print(response)

BadRequestError: Error code: 400 - {'error': {'message': 'This endpoint\'s maximum context length is 32768 tokens. However, you requested about 45602 tokens (45602 of text input). Please reduce the length of either one, or use the "middle-out" transform to compress your prompt automatically.', 'code': 400, 'metadata': {'provider_name': None}}}

In [ ]:
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPEN_ROUTER_KEY,
    model="mistralai/mistral-7b-instruct"
)

response = llm.invoke("Génère une question de QCM sur Napoléon. donne moi un JSON")
print(response)

content=' {\n\n  "question": "Quel était le titre complet de Napoléon Bonaparte lorsqu\'il était empereur des Français ?",\n  "choix": [\n    "Napoléon Ier, roi de la France",\n    "Napoléon Ier, premier consul",\n    "Napoléon Ier, empereur des Français",\n    "Napoléon Ier, roi d\'Italie"\n  ],\n  "réponse": "Napoléon Ier, empereur des Français"\n}' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 131, 'prompt_tokens': 25, 'total_tokens': 156, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/mistral-7b-instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run--bdb38260-fbfb-404f-920e-f4944dd34a47-0'


In [ ]:
from langchain.chat_models import ChatOpenAI
OPEN_ROUTER_KEY="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"

llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPEN_ROUTER_KEY,
    model="mistralai/mistral-7b-instruct"
)

In [ ]:
json_quiz = []
i = 0
for doc in context['documents']:
    i+=1
    prompt_chunk = f"""
    Tu es un expert en pédagogie et tu dois créer une question de quiz à choix multiples (QCM) à partir du texte ci-dessous.

    **Important** : tu ne dois utiliser que les informations contenues dans le texte ci-dessous.  
    **Tu ne dois en aucun cas utiliser des connaissances extérieures.**

    Contrainte :
    - La question doit porter sur une information claire et importante du texte.
    - Il doit y avoir exactement 4 choix.
    - Une seule bonne réponse.
    - Il faut que QCM porte uniquement
    - Retourne ta réponse **en JSON valide** dans ce format :

    {{
    "question": "...",
    "choices": ["...", "...", "...", "..."],
    "correct_answer": "..."
    }}

    Texte :
    \"\"\"
    {doc}
    \"\"\"
    """

    response = llm.invoke(prompt_chunk)
    json_quiz.append(response)
    print(i, 'turn')

json_quiz

1 turn
2 turn
3 turn
4 turn
5 turn
6 turn
7 turn
8 turn
9 turn
10 turn
11 turn
12 turn
13 turn
14 turn
15 turn
16 turn
17 turn
18 turn
19 turn
20 turn
21 turn
22 turn
23 turn
24 turn
25 turn
26 turn
27 turn
28 turn
29 turn
30 turn
31 turn
32 turn
33 turn
34 turn
35 turn
36 turn
37 turn
38 turn
39 turn
40 turn
41 turn
42 turn
43 turn
44 turn
45 turn
46 turn
47 turn
48 turn
49 turn
50 turn
51 turn
52 turn
53 turn
54 turn
55 turn
56 turn
57 turn
58 turn
59 turn
60 turn
61 turn
62 turn
63 turn
64 turn
65 turn
66 turn
67 turn
68 turn
69 turn
70 turn
71 turn
72 turn
73 turn
74 turn
75 turn
76 turn
77 turn
78 turn
79 turn
80 turn
81 turn
82 turn
83 turn
84 turn
85 turn
86 turn
87 turn
88 turn
89 turn
90 turn
91 turn
92 turn
93 turn
94 turn
95 turn
96 turn
97 turn
98 turn
99 turn
100 turn
101 turn
102 turn
103 turn
104 turn
105 turn
106 turn
107 turn
108 turn
109 turn
110 turn
111 turn
112 turn
113 turn
114 turn
115 turn
116 turn
117 turn
118 turn
119 turn
120 turn
121 turn
122 turn
123 turn
1

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Key limit exceeded. Manage it using https://openrouter.ai/settings/keys', 'code': 403}}

In [ ]:
json_quiz = []

for doc in range(0,20):
    prompt_chunk = f"""
    Tu es un expert en pédagogie et tu dois créer une question de quiz à choix multiples (QCM) à partir du texte ci-dessous.

    **Important** : tu ne dois utiliser que les informations contenues dans le texte ci-dessous.  
    **Tu ne dois en aucun cas utiliser des connaissances extérieures.**

    Contrainte :
    - La question doit porter sur une information claire et importante du texte.
    - Il doit y avoir exactement 4 choix.
    - Une seule bonne réponse.
    - Il faut que QCM porte uniquement
    - Retourne ta réponse **en JSON valide** dans ce format :

    {{
    "question": "...",
    "choices": ["...", "...", "...", "..."],
    "correct_answer": "..."
    }}

    Texte :
    \"\"\"
    {context['documents'][doc]}
    \"\"\"
    """

    response = llm.invoke(prompt_chunk)
    json_quiz.append(response)
    print(doc)

json_quiz

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


[AIMessage(content=' {\n  "question": "Selon le texte, qu\'est-ce que la Préhistoire représente ?",\n  "choices": [\n    "La période après l\'invention de l\'écriture",\n    "La période où les historiens font des fouilles archéologiques et étudient des objets, des documents, des récits",\n    "La période où les historiens représentent le temps par une ligne graduée : c\'est la frise chronologique",\n    "La période où l\'étude de notre passé pour mieux comprendre notre vie aujourd\'hui est faite"\n  ],\n  "correct_answer": "La période avant l\'invention de l\'écriture"\n}', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 173, 'prompt_tokens': 432, 'total_tokens': 605, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/mistral-7b-instruct', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--8dcac3d7-ee16-42c8-907b-3c5e077f6c69-0'),
 AIMessage(content=' {\n    "question": "Quel est le nom

In [ ]:
context['documents'][4]

'font souvent la guerre.  \n \n*Ils vivent dans des fermes  regroupées en village. Ce sont des \nagriculteurs, des éleveurs , pour cela ils déboisent pour former des \nchamps qui sont encore présents aujourd’hui.  \n \n*Ce sont également des artisans  habiles qui créent des outils et \ninventent le tonneau, la roue, le savon, la charrue attelée, la faux…\n5) Qui sont les Gallo-Romains ? \n \n*Les Romains, emmenés par Jules César, ont envahi \nla Gaule en l’an 52 av. JC . Les Gaulois deviennent'

In [ ]:
from langchain_community.chat_models import ChatGroq
GROQ_API_KEY="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"

llm = ChatGroq(
    model_name="mixtral-8x7b-32768",  # Tu peux aussi tester llama3
    groq_api_key=GROQ_API_KEY
)

# Exemple de prompt simple
response = llm.invoke("Génère une question de quiz avec 4 réponses sur Napoléon.")
print(response)

ImportError: cannot import name 'ChatGroq' from 'langchain_community.chat_models' (/usr/local/lib/python3.11/site-packages/langchain_community/chat_models/__init__.py)